In [133]:
import numpy as np 
import os
import pandas as pd
from PIL import Image
import slideio
import matplotlib.pyplot as plt
import cv2
import math
import tensorflow as tf
from tqdm.auto import tqdm

In [135]:
Transfer_df=pd.read_csv('../../data/100x_Transfer.csv')
HE_svs_path='../../data/svs/HE/'
stain_svs_path='../../data/svs/'
tile_path= '../../data/tiles_2/'
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")
        
def rotate_and_translate_coordinates(coords, angle_degrees, translation_x, translation_y, image_width, image_height):
    # 각도를 라디안으로 변환
    angle_rad = np.radians(angle_degrees)
  
    # 이미지 중심으로 이동
    coords_centered = coords - np.array([image_width / 2, image_height / 2])
    
    # 회전 행렬 구성
    rotation_matrix = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                               [np.sin(angle_rad), np.cos(angle_rad)]])
    # 이동
    rotated_and_translated_coordinates = coords_centered + np.array([translation_x, translation_y])
    
    # 좌표 회전
    rotated_coordinates = np.dot(rotated_and_translated_coordinates, rotation_matrix.T)
    
    
    
    # 다시 이미지 중심으로 이동
    rotated_coordinates += np.array([image_width / 2, image_height / 2])
    
    return rotated_coordinates

def rotate(xy, angle):
    x, y = xy
    x = x - 512
    y = y - 512
    rad = np.deg2rad(angle)
    x1 = x * np.cos(rad) - y * np.sin(rad)
    y1 = x * np.sin(rad) + y * np.cos(rad)
    x1 = x1 + 512
    y1 = y1 + 512
    x1=x1-x1.min()
    y1=y1-y1.min()
    x1=x1/x1.max()*1023
    y1=y1/y1.max()*1023
    x1=x1.astype(int)
    y1=y1.astype(int)
    return x1, y1

In [136]:
image_size=512
save_image_size=256
slide_tile_size=4096
for i in tqdm(range(len(Transfer_df))):
    fileName=Transfer_df.loc[i]['slide']
    stain=Transfer_df.loc[i]['stain']
    x=Transfer_df.loc[i]['x']*100
    y=Transfer_df.loc[i]['y']*100
    angle=Transfer_df.loc[i]['angle']
    he_slide = slideio.open_slide(HE_svs_path+fileName+'.svs', "SVS")
    stain_slide = slideio.open_slide(stain_svs_path+stain+'/'+fileName+'.svs', "SVS")
    he_scene = he_slide.get_scene(0)
    stain_scene = stain_slide.get_scene(0)
    he_svsWidth = he_scene.rect[2]
    he_svsHeight = he_scene.rect[3]
    stain_svsWidth = stain_scene.rect[2]
    stain_svsHeight = stain_scene.rect[3]
    inverse_Height_ratio=he_mask.shape[0]/he_svsHeight
    inverse_Width_ratio=he_mask.shape[1]/he_svsWidth
    he_thumbnali_block = he_scene.read_block((0, 0, he_svsWidth, he_svsHeight),size=(he_svsWidth//100, he_svsHeight//100))
    he_mask=cv2.threshold(cv2.cvtColor(he_thumbnali_block,cv2.COLOR_RGB2HSV)[:,:,1], 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    he_mask=cv2.morphologyEx(he_mask, cv2.MORPH_CLOSE, k)
    createDirectory(tile_path+stain+'/'+fileName+'/stain/')
    createDirectory(tile_path+stain+'/'+fileName+'/he/')
    for widthCount in range(0, int(he_svsWidth // slide_tile_size)):
        for heightCount in range(0, int(he_svsHeight // slide_tile_size)):
            point_x =int(widthCount*slide_tile_size*inverse_Width_ratio)
            point_y =int(heightCount*slide_tile_size*inverse_Height_ratio)
            tile_hw=int(slide_tile_size*inverse_Height_ratio)
            if (point_x+tile_hw)>=he_mask.shape[1]:
                point_x=he_mask.shape[1]-tile_hw-1
            if (point_y+tile_hw)>=he_mask.shape[0]:
                point_y=he_mask.shape[0]-tile_hw-1
            tile_mask_image=he_mask[point_y:point_y+tile_hw,point_x:point_x+tile_hw]/255
            if tile_mask_image.mean()>0.2:
                he_slide_block = he_scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(image_size, image_size))
                
                rectangle_coordinates = np.array([[widthCount * slide_tile_size,heightCount * slide_tile_size], [widthCount * slide_tile_size+slide_tile_size, heightCount * slide_tile_size], [widthCount * slide_tile_size, heightCount * slide_tile_size+slide_tile_size], [widthCount * slide_tile_size+slide_tile_size, heightCount * slide_tile_size+slide_tile_size]])
                result = rotate_and_translate_coordinates(rectangle_coordinates, angle, -x, -y, stain_svsWidth, stain_svsHeight)
                stain_slide_block = stain_scene.read_block((int(result[:,0].min()), int(result[:,1].min()), int(result[:,0].max())-int(result[:,0].min()), int(result[:,1].max())-int(result[:,1].min())),size=(image_size*2, image_size*2))
                he_32x_img=Image.fromarray(he_slide_block).resize((save_image_size,save_image_size))
                (cX, cY) = (image_size, image_size)
                M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
                # stain_slide_block = cv2.warpAffine(stain_slide_block, M, (image_size*2, image_size*2))
                mesh_x = np.linspace(0, image_size*2-1, image_size*2,dtype=np.int32)
                mesh_y = np.linspace(0, image_size*2-1,  image_size*2,dtype=np.int32)
                mesh_xy = np.meshgrid(mesh_x, mesh_y)
                mesh_xy = rotate(mesh_xy, angle)
                stain_slide_block=stain_slide_block[mesh_xy[1],mesh_xy[0]]
                stain_32x_img=Image.fromarray(stain_slide_block).resize((save_image_size,save_image_size))
                he_32x_img.save(tile_path+stain+'/'+fileName+'/he/'+str(widthCount * slide_tile_size)+'_'+str(heightCount * slide_tile_size)+'.jpg')
                stain_32x_img.save(tile_path+stain+'/'+fileName+'/stain/'+str(widthCount * slide_tile_size)+'_'+str(heightCount * slide_tile_size)+'.jpg')
    

  0%|          | 0/22 [00:00<?, ?it/s]

JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling value

KeyboardInterrupt: 

In [ ]:
he_mask.shape

In [ ]:
point_x